# Planner with Native Functions

This is a continuation of the [Planner Notebook](./05-using-the-planner.ipynb), which made use of Semantic Functions. Here we are exclusively using Native Functions.   A Planner can use both Semantic and Native Functions, however when using Native Functions exclusively, we are only using OpenAI services to generate the plan, and after the plan is generated, we can save it to disk, and run it as many times as we want without calling any more OpenAI endpoints.

This notebook has been verified to work with Chat Completion Service using gpt-35-turbo.

Read more about Native Functions for C# [here](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/native-functions?tabs=Csharp).

In [1]:
#r "nuget:Microsoft.SemanticKernel,0.17.230718.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;
using System.ComponentModel;
using System.Text.Json;
using Microsoft.SemanticKernel.SkillDefinition;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAIChatCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 0.17.230718.1-preview

### Creating the Plugin with Native Functions

In our example scenario, we want to answer "What documents were opened by the direct reports of the alias 'foo'?".  We are simply hardcoding a couple of `Dictionary`s that contain this information.

We will have an `SKFunction` called `GetReportsOfAlias` that takes the manager's alias and returns `IEnumerable<Employee>` - the idea is to have the planner invoke this one first as it takes in a simple string.

We will have a second `SKFunction` called `GetDocumentsOpenedBy` that takes `IEnumerable<Employee>` and returns a `Dictionary<string,List<Document>>` (mapping of employee alias to documents they've opened).

At the time of writing, Semantic Kernel doesn't support return types and input types that are `IEnumerable<...>`, so for now, we will manually call `JsonSerializer` to perform deserialization (of the input values) and serialization (of the return value.)

In [3]:

public class Employee
{
    public string Alias { get; set; }
}
public class Document
{
    public string Content { get; set; }
}

public static Dictionary<string, List<Employee>> _reports = new (){
  { "foo" , 
  new List<Employee>(){
      new Employee{Alias="report1"}, 
      new Employee{Alias="report3"}}
  },
    { "bar" , 
  new List<Employee>(){
      new Employee{Alias="report2"}, 
      new Employee{Alias="report4"}}
  }
};

public static Dictionary<string, List<Document>> _openedDocuments = new(){
  { "report1" ,
  new List<Document>(){
      new Document{Content="report1 - first document"},
      new Document{Content="report1 - second document"}}
  },
  { "report2" ,
  new List<Document>(){
      new Document{Content="report2 - first document"},
      new Document{Content="report2 - second document"}}
  },
  { "report3" ,
  new List<Document>(){
      new Document{Content="report3 - first document"},
      new Document{Content="report3 - second document"}}
  },
  { "report4" ,
  new List<Document>(){
      new Document{Content="report4 - first document"}}
  }
};

public class ReportsAndDocsPlugin{
    [SKFunction]
    public string GetReportsOfAlias([Description("The aliases to get direct reports of"), SKName("input")] string alias)
    {
        Console.WriteLine($"GetReportsOfAlias called with input: {alias}");
        //For now, we will have to serialize the return value
        return JsonSerializer.Serialize(_reports[alias]);
    }

    [SKFunction]
    public string GetDocumentsOpenedBy([Description("The documents opened by the given aliases"), SKName("input")] string employees)
    {
        //For now, we will have to deserialize the input parameter
        var reportsAsList = JsonSerializer.Deserialize<List<Employee>>(employees);
        
        var usersToOpenedDocs = new Dictionary<string, IEnumerable<Document>>();
        foreach(var r in reportsAsList){
            usersToOpenedDocs[r.Alias] = _openedDocuments[r.Alias];
        }
        
        return JsonSerializer.Serialize(usersToOpenedDocs);
    }
}

### Import the Plugin and Create the Planner

In [4]:
kernel.ImportSkill(new ReportsAndDocsPlugin(), "ReportsAndDocsPlugin");

var planner = new SequentialPlanner(kernel);

### Saving the plan

Now, let's provide the ask, then save the plan.  Alternatively, you can directly run the plan, but if you want to modify it first, you'll need to save it to disk, then load it up later.

In [9]:
var ask = "What documents were opened by the direct reports of the alias 'foo'?";

var planFile = @".\planAsJson.json";
Plan plan = await planner.CreatePlanAsync(ask);
var planAsJson = JsonSerializer.Serialize(plan, new JsonSerializerOptions { WriteIndented = true });

//Console.WriteLine(planAsJson);

File.WriteAllText(planFile, planAsJson);

### Loading and Executing the plan

Here's how you'd load the plan from disk and execute it.  

Now try this again, but go offline first, you'll notice that your plan still executes!  Neat!

In [8]:
Plan plan = Plan.FromJson(File.ReadAllText(planFile), kernel.CreateNewContext());

var result = await plan.InvokeAsync();

Console.WriteLine("Plan results:");
Console.WriteLine(result.Result.Trim());

GetReportsOfAlias called with input: foo
Plan results:
{"report1":[{"Content":"report1 - first document"},{"Content":"report1 - second document"}],"report3":[{"Content":"report3 - first document"},{"Content":"report3 - second document"}]}


### Exercise

Now, go back a few cells and change the ask to:

```
var ask = "What documents were opened by the direct reports of the aliases 'foo' and 'bar'?";
```

Why did it break? and how would you fix it?